In [1]:
import os
import json
import torch
import openai
from transformers import pipeline
from datasets import load_dataset
from collections import OrderedDict

openai.api_key= os.environ["OPENAI_API_KEY"]
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

device = torch.device("cuda:0")

2023-06-04 19:01:07.283210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 19:01:13.420421: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-04 19:01:13.428117: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-06-04 19:01:13.428134: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
dataset = load_dataset("banking77", split="test")

/home/willy/.cache/huggingface/modules/datasets_modules/datasets/banking77/9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4/banking77.py:59: FutureWarning: Dataset 'banking77' is deprecated and will be deleted. Use 'PolyAI/banking77' instead.
  warnings.warn(
Found cached dataset banking77 (/home/willy/.cache/huggingface/datasets/banking77/default/1.1.0/9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4)


In [4]:
dataset["label"]

[11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,


In [18]:
data = dataset[0:10]["text"]

In [21]:
dataset["text"]

['How do I locate my card?',
 'I still have not received my new card, I ordered over a week ago.',
 'I ordered a card but it has not arrived. Help please!',
 'Is there a way to know when my card will arrive?',
 'My card has not arrived yet.',
 'When will I get my card?',
 'Do you know if there is a tracking number for the new card you sent me?',
 'i have not received my card',
 'still waiting on that card',
 'Is it normal to have to wait over a week for my new card?',
 'How do I track my card?',
 'How long does a card delivery take?',
 "I still don't have my card after 2 weeks.  What should I do?",
 'still waiting on my new card',
 'I am still waiting for my card after 1 week.  Is this ok?',
 'I have been waiting longer than expected for my bank card, could you provide information on when it will arrive?',
 "I've been waiting longer than expected for my card.",
 "Why hasn't my card been delivered?",
 'Where is my new card? I have been waiting a week!',
 "My card still hasn't arrived af

In [4]:
intent_labels = """activate_my_card, age_limit, apple_pay_or_google_pay, atm_support, automatic_top_up, balance_not_updated_after_bank_transfer, balance_not_updated_after_cheque_or_cash_deposit, beneficiary_not_allowed, cancel_transfer, card_about_to_expire, card_acceptance, card_arrival, card_delivery_estimate, card_linking, card_not_working, card_payment_fee_charged, card_payment_not_recognised, card_payment_wrong_exchange_rate, card_swallowed, cash_withdrawal_charge, cash_withdrawal_not_recognised, change_pin, compromised_card, contactless_not_working, country_support, declined_card_payment, declined_cash_withdrawal, declined_transfer, direct_debit_payment_not_recognised, disposable_card_limits, edit_personal_details, exchange_charge, exchange_rate, exchange_via_app, extra_charge_on_statement, failed_transfer, fiat_currency_support, get_disposable_virtual_card, get_physical_card, getting_spare_card, getting_virtual_card, lost_or_stolen_card, lost_or_stolen_phone, order_physical_card, passcode_forgotten, pending_card_payment, pending_cash_withdrawal, pending_top_up, pending_transfer, pin_blocked, receiving_money, Refund_not_showing_up, request_refund, reverted_card_payment?, supported_cards_and_currencies, terminate_account, top_up_by_bank_transfer_charge, top_up_by_card_charge, top_up_by_cash_or_cheque, top_up_failed, top_up_limits, top_up_reverted, topping_up_by_card, transaction_charged_twice, transfer_fee_charged, transfer_into_account, transfer_not_received_by_recipient, transfer_timing, unable_to_verify_identity, verify_my_identity, verify_source_of_funds, verify_top_up, virtual_card_not_working, visa_or_mastercard, why_verify_identity, wrong_amount_of_cash_received, wrong_exchange_rate_for_cash_withdrawal"""

In [5]:
candidate_labels = intent_labels.split(", ")

In [14]:
pipe = pipeline("zero-shot-classification", 
                model="facebook/bart-large-mnli", 
                device=device)

In [22]:
output = pipe(data,
              candidate_labels=candidate_labels,)

In [25]:
sample = output[0]
sample

{'sequence': 'How do I locate my card?',
 'labels': ['lost_or_stolen_card',
  'verify_my_identity',
  'card_payment_not_recognised',
  'card_acceptance',
  'card_linking',
  'edit_personal_details',
  'card_arrival',
  'get_physical_card',
  'reverted_card_payment?',
  'card_not_working',
  'direct_debit_payment_not_recognised',
  'compromised_card',
  'virtual_card_not_working',
  'supported_cards_and_currencies',
  'pending_transfer',
  'activate_my_card',
  'contactless_not_working',
  'pending_card_payment',
  'getting_spare_card',
  'atm_support',
  'cash_withdrawal_not_recognised',
  'getting_virtual_card',
  'exchange_charge',
  'visa_or_mastercard',
  'why_verify_identity',
  'top_up_reverted',
  'unable_to_verify_identity',
  'card_delivery_estimate',
  'balance_not_updated_after_bank_transfer',
  'declined_transfer',
  'pending_top_up',
  'top_up_by_card_charge',
  'fiat_currency_support',
  'transfer_into_account',
  'topping_up_by_card',
  'extra_charge_on_statement',
  've

In [27]:
with open("test_output.json", "w") as f:
    json.dump(output, f, indent=4)

In [29]:
with open("/home/willy/instructod/src/IC/test_output.json", "r") as f:
    test = json.load(f)
          

In [30]:
test

[{'sequence': 'How do I locate my card?',
  'labels': ['lost_or_stolen_card',
   'verify_my_identity',
   'card_payment_not_recognised',
   'card_acceptance',
   'card_linking',
   'edit_personal_details',
   'card_arrival',
   'get_physical_card',
   'reverted_card_payment?',
   'card_not_working',
   'direct_debit_payment_not_recognised',
   'compromised_card',
   'virtual_card_not_working',
   'supported_cards_and_currencies',
   'pending_transfer',
   'activate_my_card',
   'contactless_not_working',
   'pending_card_payment',
   'getting_spare_card',
   'atm_support',
   'cash_withdrawal_not_recognised',
   'getting_virtual_card',
   'exchange_charge',
   'visa_or_mastercard',
   'why_verify_identity',
   'top_up_reverted',
   'unable_to_verify_identity',
   'card_delivery_estimate',
   'balance_not_updated_after_bank_transfer',
   'declined_transfer',
   'pending_top_up',
   'top_up_by_card_charge',
   'fiat_currency_support',
   'transfer_into_account',
   'topping_up_by_card',


In [7]:
intent_labels = """0 	activate_my_card
1 	age_limit
2 	apple_pay_or_google_pay
3 	atm_support
4 	automatic_top_up
5 	balance_not_updated_after_bank_transfer
6 	balance_not_updated_after_cheque_or_cash_deposit
7 	beneficiary_not_allowed
8 	cancel_transfer
9 	card_about_to_expire
10 	card_acceptance
11 	card_arrival
12 	card_delivery_estimate
13 	card_linking
14 	card_not_working
15 	card_payment_fee_charged
16 	card_payment_not_recognised
17 	card_payment_wrong_exchange_rate
18 	card_swallowed
19 	cash_withdrawal_charge
20 	cash_withdrawal_not_recognised
21 	change_pin
22 	compromised_card
23 	contactless_not_working
24 	country_support
25 	declined_card_payment
26 	declined_cash_withdrawal
27 	declined_transfer
28 	direct_debit_payment_not_recognised
29 	disposable_card_limits
30 	edit_personal_details
31 	exchange_charge
32 	exchange_rate
33 	exchange_via_app
34 	extra_charge_on_statement
35 	failed_transfer
36 	fiat_currency_support
37 	get_disposable_virtual_card
38 	get_physical_card
39 	getting_spare_card
40 	getting_virtual_card
41 	lost_or_stolen_card
42 	lost_or_stolen_phone
43 	order_physical_card
44 	passcode_forgotten
45 	pending_card_payment
46 	pending_cash_withdrawal
47 	pending_top_up
48 	pending_transfer
49 	pin_blocked
50 	receiving_money
51 	Refund_not_showing_up
52 	request_refund
53 	reverted_card_payment?
54 	supported_cards_and_currencies
55 	terminate_account
56 	top_up_by_bank_transfer_charge
57 	top_up_by_card_charge
58 	top_up_by_cash_or_cheque
59 	top_up_failed
60 	top_up_limits
61 	top_up_reverted
62 	topping_up_by_card
63 	transaction_charged_twice
64 	transfer_fee_charged
65 	transfer_into_account
66 	transfer_not_received_by_recipient
67 	transfer_timing
68 	unable_to_verify_identity
69 	verify_my_identity
70 	verify_source_of_funds
71 	verify_top_up
72 	virtual_card_not_working
73 	visa_or_mastercard
74 	why_verify_identity
75 	wrong_amount_of_cash_received
76 	wrong_exchange_rate_for_cash_withdrawal"""
intent_labels = intent_labels.replace("\t", "")
intent_labels = intent_labels.split("\n")

label2intent = {}
intent2label = {}
for intent in intent_labels:
    temp_split = intent.split()
    label = temp_split[0]
    intent = temp_split[1]
    label2intent[label] = intent
    intent2label[intent] = label

In [12]:
INTENTS = ""
for label in label2intent:
    INTENTS += label + " " + label2intent[label] + "\n"

In [13]:
PROMPT_TEMPLATE_1 = f"""Given the following list of INTENTS, do intent classification on the sentence provided as SENTENCE, just generate the intent and nothing more:\n
INTENTS:
{INTENTS}
SENTENCE:"""

PROMPT_TEMPLATE_2 = f"""Given the following list of INTENTS, do intent classification on the sentence provided as SENTENCE and just generate the 2 most likely intent and nothing more:\n
INTENTS:
{INTENTS}
SENTENCE:"""

PROMPT_TEMPLATE_3 = f"""Given the following list of INTENTS, do intent classification on the sentence provided as SENTENCE and just generate the 3 most likely intent and nothing more:\n
INTENTS:
{INTENTS}
SENTENCE:"""

ALL_PROMPTS = {"top1":PROMPT_TEMPLATE_1,
               "top2":PROMPT_TEMPLATE_2,
               "top3":PROMPT_TEMPLATE_3}

In [14]:
preds = {}
for i in range(2):
    cur_preds = {}

    text_input = test_data[i]
    prompt = ALL_PROMPTS["top3"] + '"' + text_input["text"] + '"'

    cur_preds["text"] = text_input["text"]
    cur_preds["gold_label"] = text_input["label"]
    cur_preds["gold_intent"] = label2intent[str(text_input["label"])]
    cur_preds["prompt"] = prompt
    

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", 
        messages=[{"role": "user", "content": prompt}]
    )
    cur_preds["pred"] = completion["choices"][0]["message"]["content"]
    preds[i] = cur_preds

In [27]:
json_filename = "banking77_top3.json"

with open("results/" + json_filename, "w") as f:
    json.dump(preds, f, indent=4)

In [25]:
with open("results/" + json_filenam, "r") as f:
    preds = json.load(f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [15]:
results = preds.copy()

In [37]:
for key, value in results.items():
    temp_result = {}
    outputs = value["pred"].split("\n")
    for output in outputs:
        if output[0] in ["1", "2", "3"]:
            temp_result["pred" + output[0]] = output.split(" ")[1]
        else:
            pass
    results[key]["intent_predicted"] = temp_result

processed_filename = json_filename[:-5] + "processed.json"
with open("results/" + processed_filename, "w") as f:
    json.dump(results, f, indent=4)

In [34]:
json_filename[:-5]

'banking77_top3'

In [16]:
results[0]

{'text': 'How do I locate my card?',
 'gold_label': 11,
 'gold_intent': 'card_arrival',
 'prompt': 'Given the following list of INTENTS, do intent classification on the sentence provided as SENTENCE and just generate the 3 most likely intent and nothing more:\n\nINTENTS:\n0 activate_my_card\n1 age_limit\n2 apple_pay_or_google_pay\n3 atm_support\n4 automatic_top_up\n5 balance_not_updated_after_bank_transfer\n6 balance_not_updated_after_cheque_or_cash_deposit\n7 beneficiary_not_allowed\n8 cancel_transfer\n9 card_about_to_expire\n10 card_acceptance\n11 card_arrival\n12 card_delivery_estimate\n13 card_linking\n14 card_not_working\n15 card_payment_fee_charged\n16 card_payment_not_recognised\n17 card_payment_wrong_exchange_rate\n18 card_swallowed\n19 cash_withdrawal_charge\n20 cash_withdrawal_not_recognised\n21 change_pin\n22 compromised_card\n23 contactless_not_working\n24 country_support\n25 declined_card_payment\n26 declined_cash_withdrawal\n27 declined_transfer\n28 direct_debit_payment_n

In [57]:
correct_count = {"pred1":0, "pred2":0, "pred3":0, "others":0}
for key, result in results.items():
    intents = result["intent_predicted"]
    gold = result["gold_intent"]
    found_gold = False
    for top, pred in intents.items():
        if pred not in intent2label:
            print(f"""This intent {pred} is not in the original intent list, it was in sample {key} from the test set and predicted as top {top[-1]}""")
        else:
            if pred == gold:
                correct_count[top] += 1
                found_gold = True
    if not found_gold:
        correct_count["others"] += 1
        print(key)
        
                
            

0
This intent getting_physical_card is not in the original intent list, it was in sample 2 from the test set and predicted as top 3
This intent getting_physical_card is not in the original intent list, it was in sample 3 from the test set and predicted as top 3


In [55]:
correct_count

{'pred1': 3, 'pred2': 1, 'pred3': 0, 'others': 1}

================================  
Observation:  

Interesting - Because in the intents there are some like "get_physical_cart" and "get-ting-_virtual_card", it creates confusion in the LLM. Inconsistency in labelling leads to inconsistency in prediction

In [49]:
intent2label

{'activate_my_card': '0',
 'age_limit': '1',
 'apple_pay_or_google_pay': '2',
 'atm_support': '3',
 'automatic_top_up': '4',
 'balance_not_updated_after_bank_transfer': '5',
 'balance_not_updated_after_cheque_or_cash_deposit': '6',
 'beneficiary_not_allowed': '7',
 'cancel_transfer': '8',
 'card_about_to_expire': '9',
 'card_acceptance': '10',
 'card_arrival': '11',
 'card_delivery_estimate': '12',
 'card_linking': '13',
 'card_not_working': '14',
 'card_payment_fee_charged': '15',
 'card_payment_not_recognised': '16',
 'card_payment_wrong_exchange_rate': '17',
 'card_swallowed': '18',
 'cash_withdrawal_charge': '19',
 'cash_withdrawal_not_recognised': '20',
 'change_pin': '21',
 'compromised_card': '22',
 'contactless_not_working': '23',
 'country_support': '24',
 'declined_card_payment': '25',
 'declined_cash_withdrawal': '26',
 'declined_transfer': '27',
 'direct_debit_payment_not_recognised': '28',
 'disposable_card_limits': '29',
 'edit_personal_details': '30',
 'exchange_charge':

In [18]:
results

[{'1': 'card_not_working',
  '2': 'lost_or_stolen_card',
  '3': 'card_delivery_estimate'},
 {'1': 'card_delivery_estimate',
  '2': 'card_arrival',
  '3': 'lost_or_stolen_card'},
 {'1': 'card_arrival',
  '2': 'card_delivery_estimate',
  '3': 'lost_or_stolen_card'},
 {'1': 'card_arrival',
  '2': 'card_delivery_estimate',
  '3': 'getting_physical_card'},
 {'1': 'card_arrival',
  '2': 'lost_or_stolen_card',
  '3': 'pending_card_payment'}]

In [7]:
dict1 = {0:1, 1:2, 2:3}
dict2 = {3:4, 4:5}

In [8]:
dict1 | dict2

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5}

In [9]:
with open("results/full_banking77_top3_0-57.json", "r") as f:
    dict57 = json.load(f)

In [15]:
len(dict57)
testdict = dict57|dict1

In [5]:
os.listdir("results/")

['.ipynb_checkpoints',
 'full_banking77_top3_319-339.json',
 'full_banking77_top3_58-117.json',
 'full_banking77_top3_118-318.json',
 'full_banking77_top3_952-3079.json',
 'banking77_top3processed.json',
 'full_banking77_top3_340-441.json',
 'full_banking77_top3_0-57.json',
 'full_banking77_top3_667-951.json',
 'full_banking77_top3_442-666.json',
 'banking77_top3.json']

In [18]:
full_results = {}
for file in os.listdir("results/"):
    if "full_banking77_top3" in file:
        with open("results/"+file, "r") as f:
            cur_results = json.load(f)
        full_results = full_results|cur_results


In [35]:
# with open("results/full_banking77_top3.json", "w") as f:
#     json.dump(full_results, f, indent=4)

In [65]:
def check_correct_format(intent2label, preds):
    for pred in preds:
        if pred not in intent2label:
            print(False)

preds = "Top_up_by_card_charge, Automatic_top_up, Top_up_limits".lower().replace(",", "").split(" ")
check_correct_format(intent2label, preds)

In [93]:
intent2label 

{'activate_my_card': '0',
 'age_limit': '1',
 'apple_pay_or_google_pay': '2',
 'atm_support': '3',
 'automatic_top_up': '4',
 'balance_not_updated_after_bank_transfer': '5',
 'balance_not_updated_after_cheque_or_cash_deposit': '6',
 'beneficiary_not_allowed': '7',
 'cancel_transfer': '8',
 'card_about_to_expire': '9',
 'card_acceptance': '10',
 'card_arrival': '11',
 'card_delivery_estimate': '12',
 'card_linking': '13',
 'card_not_working': '14',
 'card_payment_fee_charged': '15',
 'card_payment_not_recognised': '16',
 'card_payment_wrong_exchange_rate': '17',
 'card_swallowed': '18',
 'cash_withdrawal_charge': '19',
 'cash_withdrawal_not_recognised': '20',
 'change_pin': '21',
 'compromised_card': '22',
 'contactless_not_working': '23',
 'country_support': '24',
 'declined_card_payment': '25',
 'declined_cash_withdrawal': '26',
 'declined_transfer': '27',
 'direct_debit_payment_not_recognised': '28',
 'disposable_card_limits': '29',
 'edit_personal_details': '30',
 'exchange_charge':

In [201]:
def correcting_phrasing(pred, correctionfile="config/corrections.txt"):
    with open(correctionfile, "r") as f:
        lines = f.readlines()
    corrections = [correction.replace("\n", "").split(",") for correction in lines]
    for pairs in corrections:
        pred = pred.replace(pairs[0], pairs[1])
    return pred

correctionfile="config/corrections.txt"

def process_top3_intents(load_path, save_path):
    with open(load_path, "r") as f:
        results = json.load(f)
        
    full_results = results.copy()
    count = 0
    test_results = {}
    for index, result in results.items():
        topn = 1
        temp_results = {}
        pred = result["pred"].lower()
        pred = pred.replace("\n", " ")
        pred = pred.replace(",", " ")
        pred = pred.replace(".", " ")
        pred = pred.replace(":", " ")
        pred = pred.replace(")", " ")
        pred = correcting_phrasing(pred, correctionfile)
        if "_" in pred:
            pred = pred.split(" ")
            for word in pred:
                if "_" in word or word == "none":
                    temp_results["top"+str(topn)] = word
                    topn += 1
            if len(temp_results) != 3:
                count += 1
                print(f"""Predictions at index {index}: "{" ".join(pred)}" formatting isn't handled correctly\n""")

        else:
            pred = pred.split(" ")
            if pred[-1] == "":
                pred = pred[:-1]
            full_intent = ""
            for word in pred:
                if word not in ["", "1", "2", "3"]:
                    full_intent += word + "_"
                else:
                    temp_results["top"+str(topn)] = full_intent[:-1]
                    if full_intent != "":
                        topn += 1
                    full_intent = ""
            temp_results["top"+str(topn)] = full_intent[:-1]

            if len(temp_results) != 3:
                count += 1
                print(f"""Predictions at index {index}: "{" ".join(pred)}" formatting isn't handled correctly\n""")
                temp_results = {"top1":"none", "top2":"none", "top3":"none"}
        
        full_results[index]["processed_pred"] = temp_results
        
    with open(save_path, "w") as f:
        json.dump(full_results, f, indent=4)
    
    return full_results

In [200]:
load_path = "results/full_banking77_top3.json"
save_path = "results/full_banking77_top3_processed.json"

process_top3_intents(load_path, save_path)

Predictions at index 1516: "the sentence provided does not match any of the intents  as it does not mention any issue related to the banking/financial services mentioned in the intents  therefore  a list of likely intents cannot be generated" formatting isn't handled correctly

Predictions at index 1665: "the provided sentence does not match any of the given intents  please provide a valid sentence related to one of the provided intents for intent classification" formatting isn't handled correctly

Predictions at index 1898: "based on the given intents  it is unclear what the intent of the sentence is and therefore cannot be classified" formatting isn't handled correctly



{}

In [191]:
test_results["0"]

{'top1': 'card_delivery_estimate',
 'top2': 'card_about_to_expire',
 'top3': 'card_arrival'}

In [189]:
count = 0
for index, result in test_results.items():
    for topn, intent in result.items():
        if intent not in intent2label:
            count += 1
            print(index, intent, "--", full_results[index]["gold_intent"])
            
print(f"""There are {count} incorrect intent predicted""")

165 refund_not_showing_up -- extra_charge_on_statement
169 incorrect_charge_on_statement -- extra_charge_on_statement
169 refund_not_showing_up -- extra_charge_on_statement
174 dispute_charge -- extra_charge_on_statement
174 query_specific_charge -- extra_charge_on_statement
261 exchange_currency -- fiat_currency_support
293 delivery_options -- card_delivery_estimate
1017 unsupported_currency -- top_up_reverted
1037 cancelled_top_up -- top_up_reverted
1081 disputed_card_payment -- card_payment_not_recognised
1088 disputed_transaction -- card_payment_not_recognised
1094 disputed_transaction -- card_payment_not_recognised
1096 fraud_detection -- card_payment_not_recognised
1099 fraudulent_activity -- card_payment_not_recognised
1101 dispute_transaction -- card_payment_not_recognised
1103 frozen_account -- card_payment_not_recognised
1107 unauthorized_transaction -- card_payment_not_recognised
1108 disputed_transaction -- card_payment_not_recognised
1111 disputed_transaction -- card_payme

In [202]:
with open("results/full_banking77_top3_processed.json", "r") as f:
    processed = json.load(f)

In [212]:
results = {"top1":0, "top2":0, "top3":0, "others":0}
wrong_intents = {}
for index, sample in processed.items():
    gold = sample["gold_intent"]
    processed_preds = sample["processed_pred"]
    found_correct = False
    for topn, intent in processed_preds.items():
        if intent == gold:
            found_correct = True
            results[topn] += 1
    if not found_correct:
        print(index, sample["text"], gold, "--", processed_preds, "\n")
        if gold not in wrong_intents:
            wrong_intents[gold] = 1
        else:
            wrong_intents[gold] += 1
        results["others"] += 1

333 where is theft-top option? automatic_top_up -- {'top1': 'lost_or_stolen_phone', 'top2': 'lost_or_stolen_card', 'top3': 'card_delivery_estimate'} 

339 How do I make sure I don't run out of money on my card while traveling? automatic_top_up -- {'top1': 'top_up_limits', 'top2': 'cash_withdrawal_not_recognised', 'top3': 'exchange_via_app'} 

62 I found my card, I would like to reactivate it. card_linking -- {'top1': 'activate_my_card', 'top2': 'card_not_working', 'top3': 'lost_or_stolen_card'} 

64 Can I reactivate my lost card that I found this morning in my jacket pocket? card_linking -- {'top1': 'activate_my_card', 'top2': 'lost_or_stolen_card', 'top3': 'card_delivery_estimate'} 

72 Can I reactivate a card I thought I lost? card_linking -- {'top1': 'lost_or_stolen_card', 'top2': 'activate_my_card', 'top3': 'card_not_working'} 

76 Could you help me reactivate my card? It was previously lost, but I found it this morning in my jacket. card_linking -- {'top1': 'activate_my_card', 'to

In [213]:
wrong_intents

{'automatic_top_up': 5,
 'card_linking': 9,
 'card_payment_wrong_exchange_rate': 7,
 'extra_charge_on_statement': 4,
 'pending_cash_withdrawal': 8,
 'fiat_currency_support': 2,
 'card_delivery_estimate': 7,
 'top_up_reverted': 14,
 'balance_not_updated_after_cheque_or_cash_deposit': 2,
 'card_payment_not_recognised': 12,
 'why_verify_identity': 2,
 'unable_to_verify_identity': 1,
 'get_physical_card': 38,
 'visa_or_mastercard': 7,
 'topping_up_by_card': 32,
 'disposable_card_limits': 3,
 'compromised_card': 5,
 'atm_support': 1,
 'direct_debit_payment_not_recognised': 29,
 'declined_cash_withdrawal': 14,
 'pending_card_payment': 8,
 'lost_or_stolen_phone': 4,
 'request_refund': 2,
 'declined_transfer': 10,
 'Refund_not_showing_up': 1,
 'declined_card_payment': 2,
 'pending_transfer': 1,
 'terminate_account': 1,
 'card_swallowed': 2,
 'transaction_charged_twice': 1,
 'verify_source_of_funds': 3,
 'transfer_timing': 4,
 'reverted_card_payment?': 30,
 'beneficiary_not_allowed': 31,
 'tran

In [205]:
results

{'top1': 1931, 'top2': 371, 'top3': 194, 'others': 584}

In [224]:
threshold = 10

total_count = 0
for intent, count in wrong_intents.items():
    if count > threshold:
        total_count += count
        print(intent, count)
        
print(f"""Sample where more than half were wrong counted for: {total_count}""")

top_up_reverted 14
card_payment_not_recognised 12
get_physical_card 38
topping_up_by_card 32
direct_debit_payment_not_recognised 29
declined_cash_withdrawal 14
reverted_card_payment? 30
beneficiary_not_allowed 31
receiving_money 22
failed_transfer 12
transfer_into_account 17
order_physical_card 16
virtual_card_not_working 13
wrong_exchange_rate_for_cash_withdrawal 13
balance_not_updated_after_bank_transfer 14
cash_withdrawal_not_recognised 20
card_arrival 14
cancel_transfer 13
top_up_by_bank_transfer_charge 28
Sample where more than half were wrong counted for: 382


In [225]:
results_perc = {}
total = 3080
for topn, correct_count in results.items():
    results_perc[topn] = correct_count*100 / total
    
results_perc

{'top1': 62.6948051948052,
 'top2': 12.045454545454545,
 'top3': 6.298701298701299,
 'others': 18.961038961038962}

In [226]:
#Accuracy calculated without the overwhelmingly bad labels

results_perc = {}
total = 3080 - total_count
for topn, correct_count in results.items():
    if topn == "others":
        correct_count -= total_count
        results_perc[topn] = correct_count*100 / total
    else:
        results_perc[topn] = correct_count*100 / total
    
results_perc

{'top1': 71.57153446997776,
 'top2': 13.750926612305411,
 'top3': 7.190511489992587,
 'others': 7.48702742772424}

In [2]:
results_json = json.load(open("/home/willy/instructod/src/IC/results/baselines/bart-large-mnli.json", "r"))

In [ ]:
for data, output in zip(dataset, results_json):
    print(data["text"])
    gold_label = str(data["label"])
    print(gold_label, label2intent[gold_label])
    print(output["labels"][0])


In [17]:
results_json["labels"]

TypeError: list indices must be integers or slices, not str

In [21]:
golds = [label2intent[str(label)] for label in dataset["label"]]
preds = [label["labels"][0] for label in results_json]

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

average = "macro"

accuracy = accuracy_score(golds, preds)
precision = precision_score(golds, preds, average=average)
recall = recall_score(golds, preds, average=average)
f1 = f1_score(golds, preds, average=average)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.35909090909090907
Precision: 0.5366591773415444
Recall: 0.35909090909090907
F1 Score: 0.35443922165632186


/home/willy/miniconda3/envs/instructod/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
import json

def format_baseline_results(bart_results, other_results):

    reformatted_results = {}
    for idx, sample in enumerate(bart_results):
        cur_results = {} #processed_preds, gold_intent
        labels = sample["labels"]
        processed_pred = {"top1":labels[0].replace(" ", "_"),
                        "top2":labels[1].replace(" ", "_"),
                        "top3":labels[2].replace(" ", "_")}
        gold_intent = other_results[str(idx)]["gold_intent"]
        cur_results["gold_intent"] = gold_intent
        cur_results["processed_pred"] = processed_pred
        reformatted_results[str(idx)] = cur_results
    return reformatted_results

#Banking77
gpt3_top3 = json.load(open("/home/willy/instructod/src/IC/results/banking77/full_banking77_top3_gpt3.5_processed.json", "r"))
gpt3_top1 = json.load(open("/home/willy/instructod/src/IC/results/banking77/full_banking77_top1_gpt3.5_processed.json", "r"))
gpt4_top3 = json.load(open("/home/willy/instructod/src/IC/results/banking77/full_banking77_top3_gpt4_processed.json", "r"))
gpt4_top1 = json.load(open("/home/willy/instructod/src/IC/results/banking77/full_banking77_top1_gpt4_processed.json", "r"))
bart_corr = format_baseline_results(json.load(open("/home/willy/instructod/src/IC/results/baselines/bart-large-mnli-intent-correct.json", "r")), gpt3_top1)
bart = format_baseline_results(json.load(open("/home/willy/instructod/src/IC/results/baselines/bart-large-mnli.json", "r")), gpt3_top1)

#CLINC150

In [26]:
gpt3_top3["0"]

{'text': 'How do I locate my card?',
 'gold_label': 11,
 'gold_intent': 'card_arrival',
 'prompt': 'Choose the top 3 most correct intent that is expressed in given SENTENCE among the list of INTENTS provided. Generate the top 3 most likely intent and nothing more:\nINTENTS:\n0 activate_my_card\n1 age_limit\n2 apple_pay_or_google_pay\n3 atm_support\n4 automatic_top_up\n5 balance_not_updated_after_bank_transfer\n6 balance_not_updated_after_cheque_or_cash_deposit\n7 beneficiary_not_allowed\n8 cancel_transfer\n9 card_about_to_expire\n10 card_acceptance\n11 card_arrival\n12 card_delivery_estimate\n13 card_linking\n14 card_not_working\n15 card_payment_fee_charged\n16 card_payment_not_recognised\n17 card_payment_wrong_exchange_rate\n18 card_swallowed\n19 cash_withdrawal_charge\n20 cash_withdrawal_not_recognised\n21 change_pin\n22 compromised_card\n23 contactless_not_working\n24 country_support\n25 declined_card_payment\n26 declined_cash_withdrawal\n27 declined_transfer\n28 direct_debit_paymen

In [54]:
all_results = {"gpt3_top3":gpt3_top3,
               "gpt3_top1":gpt3_top1,
               "gpt4_top3":gpt4_top3,
               "gpt4_top1":gpt4_top1,
               "bart":bart,
               "bart_corr":bart_corr}

def update(dict_res, intent, setting):
    if intent not in dict_res:
        dict_res[intent] = {}
    try:
        dict_res[intent][setting] += 1
    except:
        dict_res[intent][setting] = 1
    return dict_res

def get_repartition(results):
    comparative_results = {}
    for k, v in results.items():
        gold = v["gold_intent"]
        if "top1" in v["processed_pred"]:
            top1 = v["processed_pred"]["top1"]
        else:
            top1 = None
        if "top2" in v["processed_pred"]:
            top2 = v["processed_pred"]["top2"]
        else:
            top2 = None
        if "top3" in v["processed_pred"]:
            top3 = v["processed_pred"]["top3"]
        else:
            top3 = None

        if top1 and gold == top1:
            comparative_results = update(comparative_results, gold, "gpt3_top1")
        elif top2 and gold == top2:
            comparative_results = update(comparative_results, gold, "gpt3_top2")
        elif top3 and gold == top3:
            comparative_results = update(comparative_results, gold, "gpt3_top3")

        else:
            #completely missclassed
            if gold not in comparative_results:
                comparative_results[gold] = {}
            try:
                comparative_results[gold]["gpt3_multi_fail"] += 1
            except:
                comparative_results[gold]["gpt3_multi_fail"] = 1
    return comparative_results

def print_fail(processed_results, threshold):
    count_total = 0
    total_fail_count = 0
    for k, v in processed_results.items():
        # for k1, v1 in v.items():
        if "gpt3_multi_fail" not in v:
            continue
        fail_count = v["gpt3_multi_fail"]
        if fail_count >= threshold:
            print(k, fail_count)
            count_total += 1
            total_fail_count += fail_count
    print(f"There are {count_total} misclassified for a threshold of {threshold}")
    print(f"There are {total_fail_count} total fail count for mistakes with a thrshold above {threshold}")


In [32]:
processed_results_gpt3 = get_repartition(gpt3_top3)
processed_results_gpt4 = get_repartition(gpt4_top3)
processed_results_gpt3_1 = get_repartition(gpt3_top1)
processed_results_gpt4_1 = get_repartition(gpt4_top1)
processed_results_bart = get_repartition(bart)
processed_results_bart_corr = get_repartition(bart_corr)
# print(json.dumps(processed_results, indent=2))

In [59]:
threshold = 30
print("gpt3-3")
print_fail(processed_results_gpt3, threshold)
print("===============")
print("gpt4-3")
print_fail(processed_results_gpt4, threshold)
print("===============")
print("gpt3-1")
print_fail(processed_results_gpt3_1, threshold)
print("===============")
print("gpt4-1")
print_fail(processed_results_gpt4_1, threshold)
print("===============")
print("gpt_bart")
print_fail(processed_results_bart, threshold)
print("===============")
print("gpt_bart_corr")
print_fail(processed_results_bart_corr, threshold)

gpt3-3
get_physical_card 40
topping_up_by_card 30
direct_debit_payment_not_recognised 31
reverted_card_payment? 30
There are 4 misclassified for a threshold of 30
There are 131 total fail count for mistakes with a thrshold above 30
gpt4-3
get_physical_card 40
There are 1 misclassified for a threshold of 30
There are 40 total fail count for mistakes with a thrshold above 30
gpt3-1
card_arrival 30
top_up_by_bank_transfer_charge 33
get_physical_card 40
topping_up_by_card 35
reverted_card_payment? 37
receiving_money 32
failed_transfer 30
order_physical_card 39
wrong_exchange_rate_for_cash_withdrawal 34
There are 9 misclassified for a threshold of 30
There are 310 total fail count for mistakes with a thrshold above 30
gpt4-1
fiat_currency_support 35
card_payment_not_recognised 30
get_physical_card 40
topping_up_by_card 32
There are 4 misclassified for a threshold of 30
There are 137 total fail count for mistakes with a thrshold above 30
gpt_bart
card_arrival 35
top_up_by_bank_transfer_charg

In [ ]:
def print_samples(results, intent):
    for k, v in results.items():
        if v["gold_intent"] == intent:
            print(v["text"])
            print(v["processed_pred"])
    

print_samples(gpt3_top3, "beneficiary_not_allowed")

I've tried numerous times to submit a transfer of funds. Why isn't it going through?
{'top1': 'declined_transfer', 'top2': 'transfer_not_received_by_recipient', 'top3': 'failed_transfer'}
I've transferred money before without issue but now I am encountering an error stating that it isn't possible. Why is that?
{'top1': 'declined_transfer', 'top2': 'transfer_not_received_by_recipient', 'top3': 'failed_transfer'}
I am not able to do a transfer to an account
{'top1': 'declined_transfer', 'top2': 'transfer_not_received_by_recipient', 'top3': 'pending_transfer'}
I was unable to make a transfer to another account.
{'top1': 'declined_transfer', 'top2': 'transfer_not_received_by_recipient', 'top3': 'pending_transfer'}
I had a transfer blocked
{'top1': 'declined_transfer', 'top2': 'pending_transfer', 'top3': 'cancel_transfer'}
Can you please help me with this exchange?  I am trying to get crypto and the app won't let me.
{'top1': 'exchange_via_app', 'top2': 'fiat_currency_support', 'top3': 'una

In [38]:
gpt4_top3


{'0': {'text': 'How do I locate my card?',
  'gold_label': 11,
  'gold_intent': 'card_arrival',
  'prompt': 'Choose the top 3 most correct intent that is expressed in given SENTENCE among the list of INTENTS provided. Generate the top 3 most likely intent and nothing more:\nINTENTS:\n0 activate_my_card\n1 age_limit\n2 apple_pay_or_google_pay\n3 atm_support\n4 automatic_top_up\n5 balance_not_updated_after_bank_transfer\n6 balance_not_updated_after_cheque_or_cash_deposit\n7 beneficiary_not_allowed\n8 cancel_transfer\n9 card_about_to_expire\n10 card_acceptance\n11 card_arrival\n12 card_delivery_estimate\n13 card_linking\n14 card_not_working\n15 card_payment_fee_charged\n16 card_payment_not_recognised\n17 card_payment_wrong_exchange_rate\n18 card_swallowed\n19 cash_withdrawal_charge\n20 cash_withdrawal_not_recognised\n21 change_pin\n22 compromised_card\n23 contactless_not_working\n24 country_support\n25 declined_card_payment\n26 declined_cash_withdrawal\n27 declined_transfer\n28 direct_deb

In [39]:
results = json.load(open("/home/willy/instructod/src/IC/results/clinc150/full_clinc150_top1_gpt3.5.json", "r"))

In [62]:
for k, v in gpt3_top3.items():
    if v["gold_intent"] == "topping_up_by_card":
        print(v["text"])
        print(v["pred"])

How can I top up my account with a card?
1. topping_up_by_card
2. top_up_by_card_charge
3. card_payment_not_recognised
WHERE IS MY MONEY I WAS USING MY CARD AND IT DISAPPEARED
1. declined_card_payment
2. card_not_working
3. pending_card_payment
How can I top up by card?
1. topping_up_by_card
2. top_up_by_card_charge
3. card_payment_not_recognised
What is the process for using my credit card to transfer money?
1. transfer_into_account
2. card_payment_not_recognised
3. pending_transfer
How do I top up my card using your app? I'm new to this.
1. topping_up_by_card
2. top_up_by_bank_transfer_charge
3. top_up_by_card_charge
Can friends add to my account
1. receiving_money
2. edit_personal_details
3. terminate_account
Can I top up using my car?
1. top_up_by_card_charge
2. topping_up_by_card
3. virtual_card_not_working
Why can't I see my topup in my wallet anymore?
1. pending_top_up
2. top_up_failed
3. balance_not_updated_after_bank_transfer
Use credit card to transfer money
1. transfer_into_

In [4]:

dataset = load_dataset("clinc_oos", "small", split="test")

Generating train split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5500 [00:00<?, ? examples/s]

Dataset clinc_oos downloaded and prepared to /home/willy/.cache/huggingface/datasets/clinc_oos/small/1.0.0/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1. Subsequent calls will reuse this data.


In [10]:
labels = """0 | restaurant_reviews | | 1 | nutrition_info | | 2 | account_blocked | | 3 | oil_change_how | | 4 | time | | 5 | weather | | 6 | redeem_rewards | | 7 | interest_rate | | 8 | gas_type | | 9 | accept_reservations | | 10 | smart_home | | 11 | user_name | | 12 | report_lost_card | | 13 | repeat | | 14 | whisper_mode | | 15 | what_are_your_hobbies | | 16 | order | | 17 | jump_start | | 18 | schedule_meeting | | 19 | meeting_schedule | | 20 | freeze_account | | 21 | what_song | | 22 | meaning_of_life | | 23 | restaurant_reservation | | 24 | traffic | | 25 | make_call | | 26 | text | | 27 | bill_balance | | 28 | improve_credit_score | | 29 | change_language | | 30 | no | | 31 | measurement_conversion | | 32 | timer | | 33 | flip_coin | | 34 | do_you_have_pets | | 35 | balance | | 36 | tell_joke | | 37 | last_maintenance | | 38 | exchange_rate | | 39 | uber | | 40 | car_rental | | 41 | credit_limit | | 42 | oos | | 43 | shopping_list | | 44 | expiration_date | | 45 | routing | | 46 | meal_suggestion | | 47 | tire_change | | 48 | todo_list | | 49 | card_declined | | 50 | rewards_balance | | 51 | change_accent | | 52 | vaccines | | 53 | reminder_update | | 54 | food_last | | 55 | change_ai_name | | 56 | bill_due | | 57 | who_do_you_work_for | | 58 | share_location | | 59 | international_visa | | 60 | calendar | | 61 | translate | | 62 | carry_on | | 63 | book_flight | | 64 | insurance_change | | 65 | todo_list_update | | 66 | timezone | | 67 | cancel_reservation | | 68 | transactions | | 69 | credit_score | | 70 | report_fraud | | 71 | spending_history | | 72 | directions | | 73 | spelling | | 74 | insurance | | 75 | what_is_your_name | | 76 | reminder | | 77 | where_are_you_from | | 78 | distance | | 79 | payday | | 80 | flight_status | | 81 | find_phone | | 82 | greeting | | 83 | alarm | | 84 | order_status | | 85 | confirm_reservation | | 86 | cook_time | | 87 | damaged_card | | 88 | reset_settings | | 89 | pin_change | | 90 | replacement_card_duration | | 91 | new_card | | 92 | roll_dice | | 93 | income | | 94 | taxes | | 95 | date | | 96 | who_made_you | | 97 | pto_request | | 98 | tire_pressure | | 99 | how_old_are_you | | 100 | rollover_401k | | 101 | pto_request_status | | 102 | how_busy | | 103 | application_status | | 104 | recipe | | 105 | calendar_update | | 106 | play_music | | 107 | yes | | 108 | direct_deposit | | 109 | credit_limit_change | | 110 | gas | | 111 | pay_bill | | 112 | ingredients_list | | 113 | lost_luggage | | 114 | goodbye | | 115 | what_can_i_ask_you | | 116 | book_hotel | | 117 | are_you_a_bot | | 118 | next_song | | 119 | change_speed | | 120 | plug_type | | 121 | maybe | | 122 | w2 | | 123 | oil_change_when | | 124 | thank_you | | 125 | shopping_list_update | | 126 | pto_balance | | 127 | order_checks | | 128 | travel_alert | | 129 | fun_fact | | 130 | sync_device | | 131 | schedule_maintenance | | 132 | apr | | 133 | transfer | | 134 | ingredient_substitution | | 135 | calories | | 136 | current_location | | 137 | international_fees | | 138 | calculator | | 139 | definition | | 140 | next_holiday | | 141 | update_playlist | | 142 | mpg | | 143 | min_payment | | 144 | change_user_name | | 145 | restaurant_suggestion | | 146 | travel_notification | | 147 | cancel | | 148 | pto_used | | 149 | travel_suggestion | | 150 | change_volume |"""
labels = [label.split(" | ") for label in labels.split(" | | ")]
print(labels[:3])

[['0', 'restaurant_reviews'], ['1', 'nutrition_info'], ['2', 'account_blocked']]


In [17]:
def get_label_intent_dict(labels_str:str):
    intent_labels = labels_str.replace("\t", "")
    intent_labels = labels_str.split("\n")

    label2intent = {}
    intent2label = {}
    for intent in intent_labels:
        temp_split = intent.split()
        label = temp_split[0]
        intent = temp_split[1]
        label2intent[label] = intent
        intent2label[intent] = label
        
    return label2intent, intent2label

def get_labels(labels_path):
    with open(labels_path, "r") as f:
        LABELS = f.read()
    return LABELS

def get_intents(label2intent):
    INTENTS = ""
    for label in label2intent:
        INTENTS += label + " " + label2intent[label] + "\n"
    return INTENTS


def get_prompts(prompts_path, intents):
    PROMPT = {}
    with open(prompts_path, "r") as f:
        prompts = f.readlines()
        
    for idx, prompt in enumerate(prompts):
        PROMPT["top"+str(idx+1)] = prompt.replace("{INTENTS}", intents).replace("\\n", "\n")
        
    return PROMPT

In [18]:
LABELS = get_labels("/home/willy/instructod/src/IC/config/intents_clinc.txt")
label2intent, intent2label = get_label_intent_dict(LABELS)
INTENTS = get_intents(label2intent)
ALL_PROMPTS = get_prompts("/home/willy/instructod/src/IC/config/prompts.txt", INTENTS)    

In [22]:
dataset[0]

{'text': 'how would you say fly in italian', 'intent': 61}

In [26]:
sample = "1. definition\n2. translation\n3. ingredients_list"
sample = "intent: translation"

[(idx+1, pred.split(" ")[1]) for idx, pred in enumerate(sample.split("\n"))]

[(1, 'translation')]

In [28]:
LABELS

labels = [label.split(" ")[1] for label in LABELS.split("\n")]

In [9]:
from datasets import load_dataset

clinc150 = load_dataset("clinc_oos", "small", split="test")

Found cached dataset clinc_oos (/home/willy/.cache/huggingface/datasets/clinc_oos/small/1.0.0/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1)


In [11]:
d = {}
for sample in clinc150:
    text = sample["text"]
    label = sample["intent"]
    if label in d:
        d[label] += 1
    else:
        d[label] = 0


In [12]:
d

{61: 29,
 133: 29,
 32: 29,
 139: 29,
 22: 29,
 64: 29,
 81: 29,
 128: 29,
 97: 29,
 28: 29,
 129: 29,
 29: 29,
 79: 29,
 90: 29,
 4: 29,
 103: 29,
 80: 29,
 33: 29,
 144: 29,
 77: 29,
 125: 29,
 115: 29,
 121: 29,
 3: 29,
 23: 29,
 35: 29,
 85: 29,
 20: 29,
 100: 29,
 96: 29,
 78: 29,
 11: 29,
 66: 29,
 118: 29,
 68: 29,
 145: 29,
 50: 29,
 111: 29,
 71: 29,
 101: 29,
 69: 29,
 91: 29,
 113: 29,
 13: 29,
 142: 29,
 123: 29,
 107: 29,
 149: 29,
 74: 29,
 65: 29,
 76: 29,
 119: 29,
 98: 29,
 30: 29,
 132: 29,
 1: 29,
 60: 29,
 39: 29,
 138: 29,
 95: 29,
 62: 29,
 148: 29,
 131: 29,
 146: 29,
 130: 29,
 124: 29,
 92: 29,
 54: 29,
 86: 29,
 53: 29,
 12: 29,
 134: 29,
 25: 29,
 83: 29,
 48: 29,
 51: 29,
 122: 29,
 56: 29,
 135: 29,
 87: 29,
 0: 29,
 45: 29,
 34: 29,
 18: 29,
 8: 29,
 120: 29,
 47: 29,
 38: 29,
 140: 29,
 150: 29,
 57: 29,
 41: 29,
 102: 29,
 9: 29,
 84: 29,
 89: 29,
 114: 29,
 2: 29,
 21: 29,
 137: 29,
 37: 29,
 19: 29,
 112: 29,
 70: 29,
 31: 29,
 10: 29,
 116: 29,
 136: 